# IBM Data Science Capstone

### Initially we will import all the required modules

In [171]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

#### Using requests module we will scrape the following link:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M <br>
After that we will fetch only the data.

In [172]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Using BeautifulSoup I have processed the html data which has been stored in soup variable.

In [173]:
soup = BeautifulSoup(data,'lxml')

#### Below we have use find function to find the table tag
And Then we have use <b>read_html</b> function to read the html table into Dataframe<br>
Since the function returns a list therefore we have taken the first table from the list.

In [174]:
table = soup.find('table')
df = pd.read_html(str(table))
df = df[0]
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
print(df.shape)
df.head(10)

(288, 3)


,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


#### All the rows where Borough is  Not Assigned have been droped.
#### Futhermore the Not Assigned value have been replaced with the Borough value respectively

In [175]:
df = df.replace('Not assigned',np.nan)
df.dropna(subset = ['Borough'],inplace=True)
df['Neighbourhood'] = df['Neighbourhood'].fillna(df['Borough'])
df.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [176]:
df.shape

(211, 3)

### Since we need Latitude and Longitude for respective postal code so we will be joining two Dataframes

In [177]:
lat = pd.read_csv('Geospatial_Coordinates.csv')
lat.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
lat.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Resultant DataFrame

In [178]:
result = pd.merge(df, lat,on = 'Postcode')
result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
